<a href="https://colab.research.google.com/github/nikitaoltyan/Mech-Trading-Systems/blob/main/ATR_BollingerBands_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Project cloning
!git clone https://github.com/nikitaoltyan/Mech-Trading-Systems.git
%cd Mech-Trading-Systems

Cloning into 'Mech-Trading-Systems'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 274 (delta 5), reused 0 (delta 0), pack-reused 262
Receiving objects: 100% (274/274), 1.74 MiB | 12.03 MiB/s, done.
Resolving deltas: 100% (134/134), done.
/content/Mech-Trading-Systems


In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [3]:
df = pd.read_excel("LHAG.xlsx")
df.head()

,<DATE>,<CLOSE>,<OPEN>,<HIGH>,<LOW>,<VOL>
0,"Dec 30, 2020",10.815,11.000,11.030,10.735,3.15M
1,"Dec 29, 2020",10.960,10.630,11.290,10.560,11.25M
2,"Dec 28, 2020",10.375,10.170,10.615,10.155,6.45M
3,"Dec 23, 2020",9.860,9.518,9.888,9.496,3.55M
4,"Dec 22, 2020",9.450,9.390,9.600,9.328,3.16M


In [5]:
from PlotDraw import draw_plot
draw_plot(df, 0, df.shape[0])

ATR


In [6]:
def atr(data, start, t_period = 14):
  true_range = []
  for i in range(t_period):
    tr_i = max(data['<HIGH>'][start+i]-data['<LOW>'][start+i], data['<HIGH>'][start+i]-data['<CLOSE>'][start+i], data['<CLOSE>'][start+i]-data['<LOW>'][start+i] )
    true_range.append(tr_i)
  av_true_range = sum(true_range)/t_period
  return av_true_range

In [7]:
atr(data = df, start = 0)

0.3877142857142855

Bollinger Bands

In [8]:
def bollinger_bands(data, period = 20, multiplier = 2):
  bb_data = data.copy()
  bb_data['MovingAverage'] = bb_data['<CLOSE>'].rolling(period).mean()
  bb_data['UpperBand'] = bb_data['MovingAverage'] + bb_data['<CLOSE>'].rolling(period).std() * multiplier
  bb_data['LowerBand'] = bb_data['MovingAverage'] - bb_data['<CLOSE>'].rolling(period).std() * multiplier
  return bb_data

In [9]:
bollinger_bands(data = df)

,<DATE>,<CLOSE>,<OPEN>,<HIGH>,<LOW>,<VOL>,MovingAverage,UpperBand,LowerBand
0,"Dec 30, 2020",10.815,11.000,11.030,10.735,3.15M,NaN,NaN,NaN
1,"Dec 29, 2020",10.960,10.630,11.290,10.560,11.25M,NaN,NaN,NaN
2,"Dec 28, 2020",10.375,10.170,10.615,10.155,6.45M,NaN,NaN,NaN
3,"Dec 23, 2020",9.860,9.518,9.888,9.496,3.55M,NaN,NaN,NaN
4,"Dec 22, 2020",9.450,9.390,9.600,9.328,3.16M,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1258,"Jan 08, 2016",14.920,15.060,15.140,14.840,5.90M,13.86550,14.865163,12.865837
1259,"Jan 07, 2016",15.010,14.800,15.150,14.615,8.07M,13.96900,14.993529,12.944471
1260,"Jan 06, 2016",15.160,14.910,15.410,14.900,8.04M,14.06450,15.160425,12.968575
1261,"Jan 05, 2016",14.925,14.670,14.995,14.625,6.52M,14.12175,15.273313,12.970187


In [10]:
def bb_plot(data, start = 0, end = 1):
  bb_data = bollinger_bands(data = df)
  plot = [go.Candlestick(x=[i for i in range (0, len(bb_data))][start:end],
              open=bb_data['<OPEN>'][start:end],
              high=bb_data['<HIGH>'][start:end],
              low=bb_data['<LOW>'][start:end],
              close=bb_data['<CLOSE>'][start:end])]

  fig = go.Figure(data=plot)

  fig.add_trace(go.Scatter(x = [i for i in range (0, len(data))][start:end] , y = bb_data['MovingAverage'][start:end], line = go.scatter.Line(color="gray")))
  fig.add_trace(go.Scatter(x = [i for i in range (0, len(data))][start:end] , y = bb_data['UpperBand'][start:end], line = go.scatter.Line(color="gray")))
  fig.add_trace(go.Scatter(x = [i for i in range (0, len(data))][start:end] , y = bb_data['LowerBand'][start:end], line = go.scatter.Line(color="gray")))
  fig.show()

In [ ]:
bb_plot(data = df, start = 0, end = 400)

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from Trend import find_rising_trend_lines, find_decreasing_trend_lines

In [89]:
class TradingBot:

    def __init__(self, data):
        self.data = data.dropna()
        self.low = self.data["<LOW>"]
        self.close = self.data["<CLOSE>"]
        self.high = self.data["<HIGH>"]
        self.differances = self.data["UpperBand"] - self.data["LowerBand"]
        self.commission = 0.003 #in %

        self.hyperparams = {
            "dif": 1,
            "alphaSell": 0.05,
            "betaSell": 0.05,
            "alphaBuy": 0.05,
            "betaBuy": 0.05
        }

        trendsBuy = find_rising_trend_lines(self.data, 4, 5)
        indexesBuy = self.findOpenPositionIndexes(trendsBuy, self.hyperparams["dif"])
        buyDict = {i:"Buy" for i in indexesBuy}
        trendsSell = find_decreasing_trend_lines(self.data, 4, 5)
        indexesToDelete = 0
        for trend in trendsSell:
            if trend[0] < 20:
                indexesToDelete += 1
        for _ in range(indexesToDelete):
            trendsSell.pop(0)
        indexesSell = self.findOpenPositionIndexes(trendsSell, self.hyperparams["dif"])
        sellDict = {i:"Sell" for i in indexesSell}

        self.useDictinary = {**buyDict, **sellDict}



    def useTradingSystem(self, money):
        money = money
        startMoney = money
        currentIndex = 0
        for index in sorted(self.useDictinary):
            currentMoney, _, closeIndex = self.openRightPosition(index, self.useDictinary[index], money, currentIndex,
                                                                        self.hyperparams["alphaSell"],
                                                                        self.hyperparams["betaSell"], 
                                                                        self.hyperparams["alphaBuy"], 
                                                                        self.hyperparams["betaBuy"])
            money = currentMoney
            currentIndex = closeIndex
        return (money, money-startMoney)

    
    def findOpenPositionIndexes(self, trends, dif):
        openPositionIndexes = []
        for trend in trends:
            trendStart = trend[3]
            if self.differances[trendStart] < dif:
                openPositionIndexes.append(trendStart)
        return openPositionIndexes
    

    def findCloseSellPosition(self, openIndex, money, boughtIndex, alpha, beta, lenth):
        # Проверка на возможность продать акции (что предыдущая сделка закрыта)
        if boughtIndex > openIndex:
            return (money, 0, boughtIndex)
        index = openIndex
        openPrice = self.close[openIndex]
        sharesBought = money // openPrice
        commission = round(sharesBought * openPrice * self.commission, 5)
        rest = money - (openPrice * sharesBought)
        while (openPrice/self.close[index] < (1+alpha)) and (openPrice/self.close[index] > (1-beta)):
            if lenth-1 > index:
                index += 1
            else:
                break
        commission += round(sharesBought * self.close[index] * self.commission, 5)
        profit = round(sharesBought*openPrice - sharesBought*self.close[index] - commission, 2)
        returnMoney = profit + money
        return (returnMoney, profit, index)


    def findCloseBuyPosition(self, openIndex, money, boughtIndex, alpha, beta, lenth):
        # Проверка на возможность приобрести акции (что предыдущая сделка закрыта)
        if boughtIndex > openIndex:
            return (money, 0, boughtIndex)
        index = openIndex
        openPrice = self.close[openIndex]
        sharesBought = money // openPrice
        commission = round(sharesBought * openPrice * self.commission, 5)
        rest = money - (openPrice * sharesBought)
        while (openPrice/self.close[index] > (1-alpha)) and (openPrice/self.close[index] < (1+beta)):
            if lenth-1 > index:
                index += 1
            else:
                break

        commission += round(sharesBought * self.close[index] * self.commission, 5)
        profit = round(sharesBought*self.close[index] - sharesBought*openPrice - commission, 2)
        returnMoney = profit + money
        return (returnMoney, profit, index)


    def openRightPosition(self, index, position, money, curIndex, aS, bS, aB, bB):
        if position == "Sell":
            money, profit, closeIndex = self.findCloseSellPosition(index, money, curIndex, aS, bS, self.data.shape[0])
            return (money, profit, closeIndex)
        else:
            money, profit, closeIndex = self.findCloseBuyPosition(index, money, curIndex, aB, bB, self.data.shape[0])
            return (money, profit, closeIndex)

    
    def fitBestHyperparams(self, iterations, showProcess=False):
        difs = [0, 3]
        alphasSell = [0, 0.1]
        betasSell = [0, 0.1]
        alphasBuy = [0, 0.1]
        betasBuy = [0, 0.1]
        results = {}

        for iter in range (iterations):
            dif = np.random.uniform(difs[0], difs[1])
            alphaSell = np.random.uniform(alphasSell[0], alphasSell[1])
            betaSell = np.random.uniform(betasSell[0], betasSell[1])
            alphaBuy = np.random.uniform(alphasBuy[0], alphasBuy[1])
            betaBuy = np.random.uniform(betasBuy[0], betasBuy[1])

            trendsBuy = find_rising_trend_lines(self.data, 4, 5)
            indexesBuy = self.findOpenPositionIndexes(trendsBuy, dif)
            buyDict = {i:"Buy" for i in indexesBuy}
            trendsSell = find_decreasing_trend_lines(self.data, 4, 5)
            indexesToDelete = 0
            for trend in trendsSell:
                if trend[0] < 20:
                    indexesToDelete += 1
            for _ in range(indexesToDelete):
                trendsSell.pop(0)
            indexesSell = self.findOpenPositionIndexes(trendsSell, dif)
            sellDict = {i:"Sell" for i in indexesSell}

            self.useDictinary = {**buyDict, **sellDict}

            money = 100000
            startMoney = money
            currentIndex = 0
            for index in sorted(self.useDictinary):
                currentMoney, _, closeIndex = self.openRightPosition(index, self.useDictinary[index], money, currentIndex,
                                                                            alphaSell, betaSell, alphaBuy, betaBuy)
                money = currentMoney
                currentIndex = closeIndex
            results[money-startMoney] = (dif, alphaSell, betaSell, alphaBuy, betaBuy)

            if (showProcess):
                print(f"Iter: {iter}, profit: {money-startMoney}, dif: {dif} coefs: {alphaSell, betaSell, alphaBuy, betaBuy}")
        if (showProcess):
            print(f"Max profit: {max(results)} for $100k start money was received. \nUsed alphas and betas: {results[max(results)]}")
        # Return alphaSell, betaSell, alphaBuy, betaBuy
        self.hyperparams["dif"] = results[max(results)][0]
        self.hyperparams["alphaSell"] = results[max(results)][1]
        self.hyperparams["betaSell"] = results[max(results)][2]
        self.hyperparams["alphaBuy"] = results[max(results)][3]
        self.hyperparams["betaBuy"] = results[max(results)][4]

        trendsBuy = find_rising_trend_lines(self.data, 4, 5)
        indexesBuy = self.findOpenPositionIndexes(trendsBuy, self.hyperparams["dif"])
        buyDict = {i:"Buy" for i in indexesBuy}
        trendsSell = find_decreasing_trend_lines(self.data, 4, 5)
        indexesToDelete = 0
        for trend in trendsSell:
            if trend[0] < 20:
                indexesToDelete += 1
        for _ in range(indexesToDelete):
            trendsSell.pop(0)
        indexesSell = self.findOpenPositionIndexes(trendsSell, self.hyperparams["dif"])
        sellDict = {i:"Sell" for i in indexesSell}
        self.useDictinary = {**buyDict, **sellDict}
        print("Model fitted")

In [90]:
bb = bollinger_bands(df)
bot = TradingBot(bb)

In [91]:
bot.useTradingSystem(10000)

(7422.630000000002, -2577.369999999998)

In [93]:
bot.fitBestHyperparams(512, showProcess=True)

Iter: 0, profit: -28885.639999999985, dif: 2.669293494608743 coefs: (0.03344183785615823, 0.002034152008975665, 0.0018293447217016202, 0.08654972511029727)
Iter: 1, profit: -33213.91, dif: 1.2550454037346048 coefs: (0.06670290973996593, 0.026480921757145306, 0.05090109196463384, 0.025119529285056764)
Iter: 2, profit: -60813.51000000001, dif: 1.9485509607119798 coefs: (0.031219626902836185, 0.09873896994759374, 0.018931592335024172, 0.09860943434310881)
Iter: 3, profit: 0, dif: 0.09724049652857114 coefs: (0.07296293939527275, 0.04710570788027513, 0.0514856469957529, 0.09703974664159065)
Iter: 4, profit: -46407.33000000006, dif: 2.5990417359953986 coefs: (0.015881437823172584, 0.0657143646751357, 0.053841993864622954, 0.09716483549170216)
Iter: 5, profit: -31096.160000000003, dif: 0.733151200059017 coefs: (0.03583987275075606, 0.07147672851330687, 0.0013810505484131985, 0.08900886065086354)
Iter: 6, profit: -3207.199999999997, dif: 0.5876444134842276 coefs: (0.0588493094820898, 0.0176866

In [97]:
bot.useTradingSystem(2000)

(3703.75, 1703.75)